In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import keras
import glob
import os

import warnings
warnings.filterwarnings("ignore")

In [34]:
def read_inputs():
    holidays = pd.read_csv("../data/holidays_events.csv")
    holidays = holidays[holidays["transferred"]==False]
    holidays = holidays[holidays["type"].isin(['Holiday','Additional','Bridge','Event','Transfer'])]
    
    oil = pd.read_csv("../data/oil.csv")
    oil["date"] = pd.to_datetime(oil['date'])
    oil_new = pd.DataFrame({"date":list(pd.date_range(start=oil.date.min(),end=oil.date.max()))})
    oil_new = pd.merge(oil_new,oil,how="left",on=["date"])
    oil_new['dcoilwtico'] = oil_new['dcoilwtico'].ffill()
    oil_new['dcoilwtico'] = oil_new['dcoilwtico'].bfill()

    stores = pd.read_csv("../data/stores.csv")
    transaction = pd.read_csv("../data/transactions.csv")

    return holidays, oil_new, stores, transaction

sample = pd.read_csv("../data/sample_submission.csv")
train = pd.read_csv("../data/train.csv")
test = pd.read_csv("../data/test.csv")

holidays, oil, stores, transaction = read_inputs()

In [42]:
df_train = pd.merge(train,stores[["store_nbr","cluster"]],how="left",on=["store_nbr"])

In [60]:
df_train_gpy = df_train.groupby(["date","cluster"]).agg({"sales":['sum','mean',"count"]}).reset_index()
df_train_gpy.columns = [' '.join(col).strip() for col in df_train_gpy.columns.values]
df_train_gpy = df_train_gpy[df_train_gpy['sales sum']>0]
df_train_gpy.head(2)
df_train_gpy.to_excel("../output/sales_plot.xlsx",index=False)

Index(['sales sum', 'sales mean', 'sales count'], dtype='object')